In [2]:
import requests
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import csv
import time

In [4]:
#This module will scrap data from ESPNCricinfo one element at a time
def scrap_row (driver,row,column):
    data = driver.find_element(By.XPATH,'//*[@id="ciHomeContentlhs"]/div[3]/table[3]/tbody/tr['+str(row)+']/td['+str(column)+']')
    return data.text

#This module will be used to write the scrapped data one entry (one row) at a time
def write_to_csv(Data):
    with open('ESPNCricinfo.csv', 'a', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(Data)
        f.close()

#This module is to clean the data and change data types for further analysis
def data_clean(Data):
    #Below commands are to clean the data and change data types for further analysis
    Data.insert(1,Data[0][Data[0].index('(')+1:-1])
    Data[0]=Data[0][:Data[0].index('(')-1]
    Data[8]=Data[8][2:]
    if Data[2][len(Data[2])-1]=='*':
        Data[2]=Data[2][:-1]
    Data[2:4] = [int(i) if i.isdigit() else 0 for i in Data[2:4]]
    Data[6]=float(Data[6])
    Data[7]=int(Data[7])
    Data[10]=datetime.strptime(Data[10],'%d %b %Y').date()
    return Data

In [6]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach",True)

Attributes = ["Player","Country","Runs","Balls Faced","4s","6s","Strike Rate","Inns","Opposition","Ground","Date"]
write_to_csv(Attributes) #Creates a csv file and adds the header
with open("URL for data.txt") as f:
    url = f.read()

In [ ]:
k=201 
count = 1
#The loop will cycle through all the relevant ESPNCricinfo pages (one at a time)
for page in range(1,12):
    if page==1:
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(url)
    else:
        driver = webdriver.Chrome(options=chrome_options)
        url_nextpage = url + ";page=" + str(page)      
        driver.get(url_nextpage)
    if page==11:
        k=96

    #The loop will scrap data from source, one row at a time using scrap_row()
    for row in range(1,k):
        Data = []
        for column in range(0,12):
            if column == 2 or column == 8:
                pass
            else:
                data = scrap_row(driver,row,column+1)
                Data.append(data)

        Data = data_clean(Data)
        write_to_csv(Data) #Appends to .csv
        print(f"row {count} added to .csv file")
        count = count + 1
    driver.close()
